<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [2]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-03-07 04:28:34--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  36.8MB/s    in 6.8s    

2020-03-07 04:28:41 (36.5 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [0]:
!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [0]:
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

In [0]:
# create an ImageDataGenerator object
image_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [26]:
# create image generator for training
train_generator = image_generator.flow_from_directory(
    directory='concrete_data_week4/train',
    target_size=(224, 224),
    batch_size=100,
    class_mode='categorical'
)

# create image generator for testing
valid_generator = image_generator.flow_from_directory(
    directory='concrete_data_week4/valid',
    target_size=(224, 224),
    batch_size=100,
    class_mode='categorical'
)


Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [27]:
# get model
model = Sequential()

model.add(VGG16(
    include_top=False,
    pooling='max',
    weights='imagenet'
))

model.add(Dense(2, activation='softmax'))

print(model.layers)

[<keras.engine.training.Model object at 0x7f53cd66d2e8>, <keras.layers.core.Dense object at 0x7f55f8140be0>]


In [0]:
# make vgg16 untrainable
model.layers[0].trainable = False

In [29]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [0]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
print(f"total number of train batches is: {len(train_generator)}")
print(f"total number of test batches is: {len(valid_generator)}")

total number of train batches is: 301
total number of test batches is: 96


In [32]:
vgg16_fit = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=len(train_generator),
    epochs=2,
    verbose=1,
    validation_data=valid_generator,
    validation_steps=len(valid_generator)
)

Epoch 1/2
301/301 [==============================] - 107s 354ms/step - loss: 0.2630 - acc: 0.9571 - val_loss: 0.0504 - val_acc: 0.9906
Epoch 2/2
301/301 [==============================] - 101s 335ms/step - loss: 0.0358 - acc: 0.9931 - val_loss: 0.0312 - val_acc: 0.9940


In [0]:
model.save('cracks_classifier_vgg16.h5')

In [34]:
# build resNet
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input as res_preprocess_input

# redefine the image generator
image_generator = ImageDataGenerator(
    preprocessing_function=res_preprocess_input
)

train_generator = image_generator.flow_from_directory(
    directory='concrete_data_week4/train',
    target_size=(224, 224),
    batch_size=100,
    class_mode='categorical'
)

valid_generator = image_generator.flow_from_directory(
    directory='concrete_data_week4/valid',
    target_size=(224, 224),
    batch_size=100,
    class_mode='categorical'
)

# define model
model_res = Sequential()
model_res.add(ResNet50(
    include_top=False,
    pooling='max',
    weights='imagenet'
))
model_res.add(Dense(2, activation='softmax'))

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [0]:
model_res.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [36]:
resNet50_fit = model_res.fit_generator(
    generator=train_generator,
    steps_per_epoch=len(train_generator),
    epochs=2,
    validation_data=valid_generator,
    validation_steps=len(valid_generator)
)

Epoch 1/2
301/301 [==============================] - 223s 740ms/step - loss: 0.0588 - acc: 0.9908 - val_loss: 0.0274 - val_acc: 0.9935
Epoch 2/2
301/301 [==============================] - 206s 686ms/step - loss: 0.0693 - acc: 0.9930 - val_loss: 0.0200 - val_acc: 0.9971


In [0]:
model_res.save('cracks_classifier_resNet50.h5')

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [0]:
from keras.applications.vgg16 import preprocess_input as vgg_preprocess_input
from keras.applications.resnet50 import preprocess_input as res_preprocess_input

# create two image generators based on vgg and resnet preprocessing module

vgg_image_generator = ImageDataGenerator(
    preprocessing_function=vgg_preprocess_input
)

res_image_generator = ImageDataGenerator(
    preprocessing_function=res_preprocess_input
)

In [42]:
test_generator_vgg = vgg_image_generator.flow_from_directory(
    directory='concrete_data_week4/test',
    shuffle=False,
    target_size=(224, 224)
)

test_generator_res = res_image_generator.flow_from_directory(
    directory='concrete_data_week4/test',
    shuffle=False,
    target_size=(224, 224)
)

Found 500 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [40]:
# load model
from keras.models import load_model
model_vgg = load_model("cracks_classifier_vgg16.h5")
model_res = load_model("cracks_classifier_resNet50.h5")

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:350: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [48]:
# evaluate vgg
model_vgg.evaluate_generator(
    generator=test_generator_vgg,
    steps=len(test_generator_vgg),
    verbose=1
)

16/16 [==============================] - 2s 94ms/step


[0.03300397626976519, 0.994]

In [46]:
# evaluate resNet
model_res.evaluate_generator(
    generator=test_generator_res,
    steps=len(test_generator_res),
    verbose=1
)

16/16 [==============================] - 5s 334ms/step


[0.001964567547064689, 1.0]

<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [49]:
# vgg prediction
vgg_pred = model_vgg.predict_generator(
    generator=test_generator_vgg,
    steps=len(test_generator_vgg),
    verbose=1
)

16/16 [==============================] - 5s 296ms/step


In [61]:
# prediction from vgg

import numpy as np

print(f"Output has shape of {vgg_pred.shape}")
print('\nThe first five outputs are:')
print(vgg_pred[:5])
first_five = np.argmax(vgg_pred[:5], axis=1)

print('\nThe first five class preditions are:')
print(list(map(lambda x: 'Positive' if x == 1 else 'Negative', first_five)))

Output has shape of (500, 2)

The first five outputs are:
[[9.9999988e-01 6.2824533e-08]
 [9.9551070e-01 4.4893781e-03]
 [5.9140706e-01 4.0859288e-01]
 [1.0000000e+00 7.8640134e-09]
 [9.9887115e-01 1.1288804e-03]]

The first five class preditions are:
['Negative', 'Negative', 'Negative', 'Negative', 'Negative']


In [62]:
# prediction from resNet
res_pred = model_res.predict_generator(
    generator=test_generator_res,
    steps=len(test_generator_res),
    verbose=1
)


16/16 [==============================] - 5s 312ms/step


In [63]:
# res prediction

print(f"Output has shape of {res_pred.shape}")
print('\nThe first five outputs are:')
print(res_pred[:5])
first_five = np.argmax(res_pred[:5], axis=1)

print('\nThe first five class preditions are:')
print(list(map(lambda x: 'Positive' if x == 1 else 'Negative', first_five)))

Output has shape of (500, 2)

The first five outputs are:
[[9.9946326e-01 5.3673587e-04]
 [9.9914324e-01 8.5668219e-04]
 [9.9973017e-01 2.6978814e-04]
 [9.9866652e-01 1.3334575e-03]
 [9.9960369e-01 3.9627909e-04]]

The first five class preditions are:
['Negative', 'Negative', 'Negative', 'Negative', 'Negative']


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).